In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import mglearn

from IPython.display import display

%matplotlib inline

In [154]:
train_data = pd.read_csv('titanic/train.csv', index_col='PassengerId')
test_data = pd.read_csv('titanic/test.csv', index_col='PassengerId')

In [84]:
print(train_data.shape)
print(test_data.shape)

(891, 11)
(418, 10)


Съединявам train с test, за да добавям колони паралелно и в двата сета ( знам че първите 891 реда са от train)

In [155]:
combo = train_data.append(test_data)

За титлите използвам кода от лекциите.

In [156]:
def make_titles(data):
    data['Title'] = data.Name.str.extract('([A-Za-z]+)\.', expand=False)
    data.loc[data.Title == 'Mlle', 'Title'] = 'Miss'
    data.loc[data.Title == 'Mme', 'Title']  = 'Mrs'
    data.loc[data.Title == 'Ms', 'Title']   = 'Miss'

    rare_titles = ['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Don', 'Jonkheer', 'Capt', 'Lady', 'Sir']
    data.Title = data.Title.replace(rare_titles, 'Rare')

За възрастите също.

In [157]:
def fill_ages_by_title(data):
    age_by_title = data.groupby('Title').Age.mean()
    data.loc[data.Age.isnull() & (data.Title == 'Mr'), 'Age'] = age_by_title['Mr']
    data.loc[data.Age.isnull() & (data.Title == 'Mrs'), 'Age'] = age_by_title['Mrs']
    data.loc[data.Age.isnull() & (data.Title == 'Miss'), 'Age'] = age_by_title['Miss']
    data.loc[data.Age.isnull() & (data.Title == 'Master'), 'Age'] = age_by_title['Master']
    data.loc[data.Age.isnull() & (data.Title == 'Rare'), 'Age'] = age_by_title['Rare']

Запълвам embarked с S, както на лекцията.

In [158]:
def fill_embarked(data):
    data.Embarked = data.Embarked.fillna('S')

In [159]:
def fill_fare(data):
    data.Fare = data.Fare.fillna(data.Fare.mean())

След малко ровене в интернет установих, че първата буква от кабината означава палубата. Тъй като палубите са на различни нива в кораба, това може да окаже влияние върху крайния резултат. Където няма въведена кабина пиша N.

In [160]:
def get_decks(data):
    data["deck"] = data.Cabin.str[0]
    data["deck"] = data.deck.fillna("N")

Резултата малко се подобрява като се разбие размера на семейството на малко и голямо семейство.

In [161]:
def bin_family(data):
    family_size = data['Parch'] + data['SibSp'] + 1
    data['family_size_one'] = family_size == 1
    data['family_size_small'] = ((2 <= family_size) & (family_size <= 4))
    data['family_size_big'] = 5 <= family_size

Голямо влияние върху резултата в kaggle оказа разделянето на вързастта. След известно натъкмяване избрах [0, 18, 36, 48, 60, 81] за кофи

In [175]:
def bin_age(data):
    data["Age"] = pd.cut(data.Age, bins=[0, 18, 36, 48, 60, 81], labels=[0, 1, 2, 3, 4])
    age_dummied = pd.get_dummies(data["Age"], prefix="Age")
    data.drop("Age", axis=1, inplace=True)
    data = pd.concat([data, age_dummied], axis=1)
    return data

Извличам буквите от началото на билета като фийчър.

Не знам какво точно означават те, но реших да пробвам дали ще окажат влияние върху резултата. Оказа се, че оказват доста влияние.

In [163]:
def manage_ticket(data):
    data["ticket_code"] = data.Ticket.str.replace("[0-9]|[/., ]", "")
    data.ticket_code = data.ticket_code.fillna("NONE")

In [185]:
def engineer_data(data_in):
    data = data_in.copy()
    fill_embarked(data)
    fill_fare(data)
    make_titles(data)
    get_decks(data)
    manage_ticket(data)
    fill_ages(data)
    bin_family(data)
    data = bin_age(data)
    data = data.drop(["Cabin", "Ticket", "Name", 'SibSp', 'Parch'], axis=1)
    data = pd.get_dummies(data)
    return data

In [187]:
whole_dataset_engineered = engineer_data(combo)
train_engineered = whole_dataset_engineered[:891]
test_engineered = whole_dataset_engineered[891:]

In [188]:
X = train_engineered.drop("Survived", axis=1)
y = train_engineered["Survived"]

In [ ]:
С известно натъкмяване на хиперпараметрите постигнах този резултат:

In [204]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0, max_depth=10, n_estimators=100, min_samples_split=15)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)
model.fit(X_train, y_train)
print("Train score ", model.score(X_train, y_train))
print("Test score ",  model.score(X_test, y_test))


Train score  0.868263473054
Test score  0.847533632287


In [202]:
transformed_test_data = test_engineered.drop("Survived", axis=1)
model = RandomForestClassifier(random_state=0, max_depth=10, n_estimators=100, min_samples_split=15)
model.fit(X, y)
predictions = model.predict(transformed_test_data)

frame = pd.DataFrame({
    'PassengerId': pd.read_csv('titanic/test.csv').PassengerId,
    'Survived': predictions.astype(int)
})
frame = frame.set_index('PassengerId')
frame.to_csv('~/Desktop/predictions.csv')
frame.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


Резултата в kaggle е 

<img src="img/kaggle_result.png">